First thing we need to do is read in our credentials for the twitter API from a separate document

In [1]:
with open("Credentials.txt", "r") as f:
    content = f.readlines()
content = [x.strip("\n") for x in content]

Now we can log into the twitter API. We also set a notification for python to stop if it hits the rate limit and wait until the 15 min time period has passed until we resume. This is helpful so we don't get blacklisted

In [3]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = '{}'.format(content[2])
consumer_secret = '{}'.format(content[3])
access_token = '{}'.format(content[4])
access_secret = '{}'.format(content[5])
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
#simple example to make sure we are in and everything is working ok
for status in tweepy.Cursor(api.home_timeline).items(5):
    print(status.text)

PCI SSC has introduced new software security standards as part of a new PCI Software #Security Framework:

→ PCI Se… https://t.co/NDAGHps7RS
It may be snowy and cold out, but we’ve got the Barrel Aged beers to warm you up! 6 of the barrel aged beauties fro… https://t.co/sPXkyvtiyY
RT @AmeliaMN: Introducing the Tidyverse Developer Day, @hadleywickham says the goal is for us to mostly be "typing R code." https://t.co/TY…
It’s Bulls vs. Heat tonight and we are partnering with @BMOHarrisBank for Pride Night! Can’t make it to the game? R… https://t.co/6uKxf4JdqH
ICYMI: A default #Cisco configuration allows full admin access to unauthenticated attackers. 

https://t.co/RSVliUMgwf


Now there are two approaches to monitoring twitter. Approach one simply looks for a keyword, username, hashtag, etc. over the last 7 day period and returns the number of results you asked for. Keep in mind the rate limit. Also this method of searching is not full proof. i.e. it can miss some results occassionally as noted in the documentation. Below I just show an example of searching for a keyword

In [5]:
term ='basketball' #search for specific words
num_tweets = 5
messages = []

#tweet mode extended is necessary to return the entire text if tweet is over 140 chars
for tweet in tweepy.Cursor(api.search, q=term, tweet_mode="extended").items(num_tweets):
    # Process a single status
    #print("Tweet ID:", tweet.id)
    if "retweeted_status" in tweet._json:
        print('Tweeted by ', tweet.user.screen_name)
        print("Created at ",tweet.created_at)
        print("Location: ",tweet.source)
        print('Tweet Text: RT ', tweet._json["retweeted_status"]["full_text"])
        messages.append(tweet._json["retweeted_status"]["full_text"])
    else:
        print('Tweeted by ', tweet.user.screen_name)
        print("Created at ",tweet.created_at)
        print("Location: ",tweet.source)
        print('Tweet Text: ', tweet.full_text)
        messages.append(tweet.full_text)     

Tweeted by  _LHSAthletics
Created at  2019-01-19 17:56:27
Location:  Twitter Web Client
Tweet Text: RT  2019 Boys Basketball Video Recap: Lowell 76, Central Catholic 68 https://t.co/65uMUYHOUH - Great work by @ConveyWEEI @lowellhighbball @kingjv19 @_LHSAthletics https://t.co/XQSpiToMf7
Tweeted by  DanimalJakes
Created at  2019-01-19 17:56:24
Location:  Twitter for iPhone
Tweet Text:  With an hour before tip, read up on one of today’s biggest games! https://t.co/hFeoTLd8D6
Tweeted by  CGTrojanHoops
Created at  2019-01-19 17:56:23
Location:  Twitter for iPhone
Tweet Text: RT  Yeah, they’re pretty good:

https://t.co/vRgxgYKZxL https://t.co/mVCqBY4oRj
Tweeted by  thequeenbro
Created at  2019-01-19 17:56:23
Location:  Twitter for iPhone
Tweet Text: RT  You have on a football jersey on the basketball court for what reason exactly? https://t.co/ehq3k0dvzu
Tweeted by  lsp37
Created at  2019-01-19 17:56:23
Location:  TweetDeck
Tweet Text:  LIVE NOW: @UCAP_Pointers @ @MassBayBucs @njcaa men's #

Now the second approach is for monitoring realtime. Similar to above we can search for multiple criteria but the example below will just show searching on keywords for an example and then writing to a txt file.

In [6]:
import csv
ex = []
#set up live stream on twitter
class MyStreamListener(tweepy.StreamListener):

    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0

    def on_status(self, status):
        self.num_tweets += 1
        try:
            print(status._json["retweeted_status"]["extended_tweet"]["full_text"])
        except KeyError:
            try:
                print(status._json["extended_tweet"]["full_text"])
            except KeyError:
                try: 
                    print(status._json["retweeted_status"]["full_text"])
                except KeyError:
                    print(status._json["text"])
        #write the data to a file for future analysis
        with open('LiveStreamResults.txt', 'a', encoding = "utf8") as f: #append each row of data to txt file
            writer = csv.writer(f)
            try:
                writer.writerow([status.author.screen_name, status.created_at, status._json["retweeted_status"]["extended_tweet"]["full_text"]])
            except KeyError:
                try:
                    writer.writerow([status.author.screen_name, status.created_at, status._json["extended_tweet"]["full_text"]])
                except KeyError:
                    try:
                        writer.writerow([status.author.screen_name, status.created_at, status._json["retweeted_status"]["text"]])
                    except KeyError:
                        writer.writerow([status.author.screen_name, status.created_at, status.text])
        if self.num_tweets > 2:
            return False
    #break the code if we hit a rate limit
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

In [7]:
keywords = ["basketball"]
myStream.filter(track=keywords)

RT @SoChasAthletics: Girls Basketball: SC 52 Capital 37 - FINAL #wvgirlsbb
Thank you to the Houston Marathon Foundation for sponsoring our 5k run! The kids did awesome!
My basketball girls ran with me, and we maintained a 9:01 pace in the wind. They motivated each other the ENTIRE way.ready for the Aramco Half Marathon tomorrow! #wearehouston5k #manvel https://t.co/wboJuuthvs
@2020fight And this &gt; Covington Catholic basketball star (18) charged with raping and sodomizing his girlfriend last month. 
https://t.co/Rcv01BPScU
